# Agent 에 Memory 추가

현재 챗봇은 과거 상호작용을 스스로 기억할수 없어 일관된 다중 턴 대화를 진행하는데 제한이 있다.

때문에 memory 를 추가하여 과거 상호작용을 기억할 수 있도록 한다

---

pre-built 가 되어있는 `ToolNode`와 `tools_condition`을 활용한다.

1. [ToolNode](https://langchain-ai.github.io/langgraph/reference/prebuilt/#langgraph.prebuilt.chat_agent_executor.create_react_agent) : 도구 호출을 위한 노드
2. [tools_condition](https://langchain-ai.github.io/langgraph/reference/prebuilt/#langgraph.prebuilt.tool_node.tools_condition) : 도구 호출 여부에 따른 조건 분기

우리의 챗봇은 이제 도구를 사용하여 사용자 질문에 답할 수 있지만, 이전 상호작용의 context를 기억하지 못한다. 이는 mutiturn 대화에 제한이 있음을 의미한다.

`LangGraph`는 persistent checkpointing 을 지원하여 이전 상호작용의 context를 기억할 수 있다.

그래프를 컴파일 할 때 `checkpointer`를 제공하고, 그래프를 호출할 때 `thread_id`를 제공하면, `LangGraph`는 각 단계 후 **상태를 자동으로 저장**한다.

동일한 `thread_id`를 사용하여 그래프를 다시 호출하면, 그래프는 저장된 상태를 로드하여 챗봇이 이전에 중단한 지점에서의 대화를 이어갈 수 있게 한다.

checkpointing은 LangChain의 메모리 기능보다 훨씬 강력하다. 


In [1]:
from dotenv import load_dotenv
load_dotenv()

True

너무 앞서 나가기 전에, multi-turn 대화를 가능하게 하기 위해 checkpointing을 추가해보도록하자.

`MemorySaver` checkpointer를 생성한다.

In [2]:
from langgraph.checkpoint.memory import MemorySaver

# 메모리 저장소 생성
memory = MemorySaver()

우리는 `in-memory checkpointer`를 사용할 것이다. 

하지만 프로덕션 단계에서는 이를 `SqliteSaver` 또는 `PostgresSaver` 로 변경하고 자체 DB에 연결이 가능하다.
